# UR5 Simulation forward kinematics
**Author** Eduardo Gomez

Using the Denavit-Hartenberg parameters of the UR5, program the forward kinematics

In [1]:
# Import libraries
import zmqRemoteApi
import numpy as np
from time import sleep
from pathlib import Path
import pandas as pd

## Setup Data

Here we will set up the data for the Denavit-Hartenberg parameters. The data was from: ...

In [2]:
df = pd.read_csv("../denavit-generated.csv")
print(df.head())

        d  theta       a  alpha
0  0.0661   90.0  0.0000  -90.0
1  0.0000    0.0  0.4251    0.0
2  0.0000    0.0  0.3922    0.0
3  0.0397  -90.0  0.0000  -90.0
4  0.0492   90.0  0.0000  -90.0


In [3]:
# First one I think is wrong



The matriz transformations can be expressed like these: 

$$
H_i^{i-1} = Rot_z(\theta_i)T(0,0,d_i)T(a_i, 0, 0)Rotx(\alpha_i)
$$

Esto se lee como una rotacion $\theta$ en $z$, una translacion $d$ en $z$, translacion $a$ en $x$ y rotacion $\alpha$ en $x$,

Here we'll define some utility functions

In [4]:
from collections.abc import Callable
import numpy as np
from functools import wraps


def deg_args(f: Callable):
    """
    Transforms all the arguments, assuming to be deg, to rads
    """

    @wraps(f)
    def wrapper(*args, **kwargs):
        new_args = [np.deg2rad(arg) for arg in args]
        new_kwargs = {key: np.deg2rad(value) for key, value in kwargs.items()}
        return f(*new_args, **new_kwargs)

    return wrapper


@deg_args
def rotation_3d_deg(yau_z : float = 0, theta_y : float = 0, phi_x : float = 0 ):
    """
    Returns the rotation matrix in 2d, given the angles
    The angles are calculated assuming the axis is the axis of rotation
    """
    R_x = np.array(
        [
            [1, 0, 0],
            [0, np.cos(phi_x), -np.sin(phi_x)],
            [0, np.sin(phi_x), np.cos(phi_x)],
        ]
    )

    R_y = np.array(
        [
            [np.cos(theta_y), 0, np.sin(theta_y)],
            [0, 1, 0],
            [-np.sin(theta_y), 0, np.cos(theta_y)],
        ]
    )

    R_z = np.array(
        [
            [np.cos(yau_z), -np.sin(yau_z), 0],
            [np.sin(yau_z), np.cos(yau_z), 0],
            [0, 0, 1],
        ]
    )

    return R_z @ R_y @ R_x, {"R_z": R_z, "R_y": R_y, "R_x": R_x}


def add_translation_to_rotation_3d(rotation_3d: np.ndarray, p: np.ndarray):
    return np.block([[rotation_3d, np.reshape(p, (3, 1))], [np.zeros((1, 3)), 1]])


def get_homogenous_point_3d(p: np.ndarray):
    return np.block([p, 1])


def mul_homogenous_matrixes(transform_matrices: list[np.ndarray]) -> np.ndarray:
    """
    Multiplies a list of homogeneous matrixes, from right to left
    """
    return reduce(lambda prev, next: next @ prev, transform_matrices[::-1])



In [5]:
# this is the identity matrix for rotation
NO_ROTATION_MATRIX, _ = rotation_3d_deg(0, 0, 0)
NO_TRANSLATION_VEC = np.array([0,0,0])


def get_homogenous_mat_for_joint(theta : float, alpha : float, a : float, d : float):
    """
    Esto se lee como una rotacion $\theta$ en $z$, una translacion $d$ en $z$, translacion $a$ en $x$ y rotacion $\alpha$ en $x$,
    """
    rotation_z, _ = rotation_3d_deg(yau_z=theta)
    rotation_z = add_translation_to_rotation_3d(rotation_z, NO_TRANSLATION_VEC)
    rotation_x, _ = rotation_3d_deg(phi_x=alpha)
    rotation_x = add_translation_to_rotation_3d(rotation_x, NO_TRANSLATION_VEC)
    
    
    translation_x = add_translation_to_rotation_3d(NO_ROTATION_MATRIX, np.array([a, 0, 0]))
    translation_z = add_translation_to_rotation_3d(NO_ROTATION_MATRIX, np.array([0, 0, d]))
    
    H = rotation_z @ translation_z @ translation_x @ rotation_x
    return H


In [6]:
H1 = get_homogenous_mat_for_joint(theta=60, d=0, a=155, alpha=0)
H2 = get_homogenous_mat_for_joint(theta=-30, d=0, a=3, alpha=0)


HT = H1 @ H2

print(np.round(HT, 2))
print(np.round(HT[0:3,3], 2))


[[  0.87  -0.5    0.    80.1 ]
 [  0.5    0.87   0.   135.73]
 [  0.     0.     1.     0.  ]
 [  0.     0.     0.     1.  ]]
[ 80.1  135.73   0.  ]


In [7]:
print(df)

        d  theta       a  alpha
0  0.0661   90.0  0.0000  -90.0
1  0.0000    0.0  0.4251    0.0
2  0.0000    0.0  0.3922    0.0
3  0.0397  -90.0  0.0000  -90.0
4  0.0492   90.0  0.0000  -90.0


In [8]:
from functools import reduce


H_mats = [
    get_homogenous_mat_for_joint(theta=row["theta"], a=row["a"], alpha=row["alpha"], d=row["d"])
    for _, row in df.iterrows()
 ]

H_total = mul_homogenous_matrixes(H_mats)

print(np.round(H_total, 4))

final = H_total @ np.array([0,0,0.0085, 1])

print(final)


[[ 1.      0.      0.     -0.0397]
 [ 0.     -1.     -0.      0.8665]
 [ 0.     -0.     -1.      0.0661]
 [ 0.      0.      0.      1.    ]]
[-0.0397  0.8665  0.0576  1.    ]


In [9]:

def print_rounded(m):
    print(np.round(m, 4))













## Setup simulation

To set up the simulation, open the `./sim/ur5-simulation.ttt` file. Then run the following code: 

> If there is an error, either the file is not opened or another script is controlling it

In [10]:
# Create a client to get connected to the zmqremoteApi sernver from CoppeliaSim
client = zmqRemoteApi.RemoteAPIClient()

# Get the remote object 'sim'
sim = client.getObject('sim')


Here, we set up the id for controlling the robot

In [11]:
UR5_sim_obj_id = "/UR5"
base_id = sim.getObject(UR5_sim_obj_id)

# the last joint is the rotor for the end effector
# does not affect the final position
no_joints = 6
joint_sim_names = [None for _ in range(6)]

joint_sim_names[0] = f"{UR5_sim_obj_id}/joint"

for i in range(1,len(joint_sim_names)):
    link_joint_str = "/link/joint" * i
    joint_sim_names[i] = f"{UR5_sim_obj_id}{link_joint_str}"

connection_name = f"{UR5_sim_obj_id}/connection"

In [12]:
# Assert that names have id. Error is thrown if id doesnt exists
joint_id = [
    sim.getObject(name) for name in joint_sim_names
]
connection_id = sim.getObject(connection_name)

In [13]:
def reset_arm():
    """
    Resets the arm position to 0 deg
    """
    for j in joint_id:
        sim.setJointTargetPosition(j, np.deg2rad(0))

In [24]:
sim.startSimulation()
reset_arm()

In [15]:
reset_arm()

In [16]:

print(sim.getObjectPosition(connection_id, sim.handle_world))
angles = [0 for _ in range(6)]
# angles[1] = 90
angles = [np.deg2rad(a) for a in angles]
assert len(angles) == no_joints

for a, j in zip(angles, joint_id):
    sim.setJointTargetPosition(j, a)

sleep(1)

for idx, j in enumerate(joint_id):
    coord = sim.getObjectPosition(j, sim.handle_world)
    print(f"j{idx}: {np.round(coord,4)}")

coord = sim.getObjectPosition(connection_id, sim.handle_world)
print(f"cn: {np.round(coord,4)}")



[-0.1850380073166695, -1.4272111220043563e-05, 0.9780369442541664]
j0: [ 0. -0.  0.]
j1: [-0.0703 -0.      0.066 ]
j2: [-0.0703 -0.      0.4911]
j3: [-0.0704 -0.      0.8833]
j4: [-0.1101 -0.      0.9289]
j5: [-0.0957 -0.      0.978 ]
cn: [-0.185 -0.     0.978]


Here we stop the simulation

In [17]:
print(df)

        d  theta       a  alpha
0  0.0661   90.0  0.0000  -90.0
1  0.0000    0.0  0.4251    0.0
2  0.0000    0.0  0.3922    0.0
3  0.0397  -90.0  0.0000  -90.0
4  0.0492   90.0  0.0000  -90.0


In [45]:
sim.startSimulation()
reset_arm()

In [46]:

print(len(H_mats))
print(len(joint_id))

for idx in range(len(H_mats)-1, -1, -1):
    print(f"Putting {5 - idx}")
    sim.setObjectMatrix(joint_id[idx + 1], joint_id[idx], H_mats[idx].flatten().tolist())
    sleep(3)
    






5
6
Putting 1
Putting 2
Putting 3
Putting 4
Putting 5


In [19]:
reset_arm()

In [44]:
sim.stopSimulation()

1